# Dummy Provider Example and High Volume Robustness Testing

This notebook has two purposes: 

- Demostrate the dummy feedback function provider which behaves like the
  huggingface provider except it does not actually perform any network calls and
  just produces constant results. It can be used to prototype feedback function
  wiring for your apps before invoking potentially slow (to run/to load)
  feedback functions.

- Test out high-volume record and feedback computation. To this end, we use the
  custom app which is dummy in a sense that it produces useless answers without
  making any API calls but otherwise behaves similarly to real apps, and the
  dummy feedback function provider.

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
%matplotlib

import time
from matplotlib import pyplot as plt

from ipywidgets import widgets
from matplotlib.ticker import StrMethodFormatter

from trulens_eval.utils.threading import TP
from IPython.display import clear_output
from threading import Thread
import matplotlib.dates as mdates

from datetime import datetime

plt.ion()

class TPMonitor():
    def __init__(self):
        self.tp = TP()
        self.qsize_points = []
        self.dtime_points = []
        self.completed_points = []
        self.timedout_points = []

        fig, ax = plt.subplots(1,1)

        self.fig = fig
        self.ax = ax
        self.ax1 = ax
        self.ax2 = ax.twinx()

        self.ax1.set_ylabel("Task Count")
        self.ax1.set_yscale("symlog", base=2)
        self.ax2.set_ylabel("Percent Full")

        self.ax1.xaxis.set_major_formatter(StrMethodFormatter("T{x:0.2f}"))
        self.ax1.yaxis.set_major_formatter(StrMethodFormatter("{x:0.0f}"))
        self.ax2.yaxis.set_major_formatter(StrMethodFormatter("{x:0.2f}%"))

        # self.fig.gca().

        self.widget = widgets.Output()
        self.widget.append_display_data(self.fig)

        # self.running = True
        # self.thread = Thread(target=self.run)

    # def start(self):
    #     self.thread.start()

    def stop(self):
        self.running = False

    def update_figure(self):
        now = datetime.now()

        self.qsize_points.append(100 * self.tp.futures.qsize() / self.tp.futures.maxsize)
        self.dtime_points.append(now)
        self.completed_points.append(self.tp.completed_tasks)
        self.timedout_points.append(self.tp.timedout_tasks)
        
        self.qsize_points = self.qsize_points[-100:]
        self.dtime_points = self.dtime_points[-100:]
        self.completed_points = self.completed_points[-100:]
        self.timedout_points = self.timedout_points[-100:]
        
        ax1 = self.ax1
        ax2 = self.ax2

        for art in ax1.lines + ax1.collections + ax2.lines + ax2.collections:
            art.remove()
        
        sec_points = [(dt-now).total_seconds() for dt in self.dtime_points]
        
        ax1.plot(sec_points, self.qsize_points, color="orange", label="% Queue fullness")
        ax2.plot(sec_points, self.completed_points, color="green", label="# Completed")
        ax2.plot(sec_points, self.timedout_points, color="red", label="# Timedout")
        
        self.fig.legend()
        self.fig.show()

        # self.ax.set_xticklabels(label_points)

        # self.ax.set_ylim(-5, self.tp.futures.maxsize+5)
        # self.ax.set_xlim(0, 100)

        # self.widget.clear_output(wait=True)
        # self.widget.append_display_data(self.fig)

    def run(self):
        while self.running:
            time.sleep(0.1)

In [ ]:
# tpm = TPMonitor()
# tpm.start()

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp

from trulens_eval import Feedback
from trulens_eval import Tru
from trulens_eval.feedback.provider.hugs import Dummy
from trulens_eval.tru_custom_app import TruCustomApp

tru = Tru()

tru.reset_database()

tru.start_dashboard(
    force = True,
    _dev=Path().cwd().parent.parent.resolve()
)

In [ ]:
f_dummy1 = Feedback(Dummy().positive_sentiment).on_output()

# Create custom app:
ca = CustomApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
    main_method = ca.respond_to_query,
    feedbacks=[f_dummy1]
)

In [ ]:
from tqdm.auto import tqdm

# Sequential app invocation.

from time import sleep

# outs = widgets.Output()
# display(widgets.VBox([tpm.widget, outs]))

if False:
    for i in tqdm(range(10000)):
        """
        if i % 20 == 0:
            tpm.update_figure()

            tpm.widget.clear_output(wait=True)
            with tpm.widget:
                display(tpm.fig)
        """

        #with outs:
        with ta as recorder:
            res = ca.respond_to_query(f"hello {i}")

        rec = recorder.get()
        assert rec is not None

In [ ]:
from tqdm.auto import tqdm

# Parallel app invocation.

from time import sleep

#outs = widgets.Output()
#display(widgets.VBox([tpm.widget, outs]))

def run_query(q):

    with ta as recorder:
        res = ca.respond_to_query(q)

    rec = recorder.get()
    assert rec is not None

    return f"run_query {q} result"

tp = TP()

for i in tqdm(range(100)):
    print(tp.completed_tasks, len(tp.futures), end="\r")
    """
    if i % 20 == 0:
        tpm.update_figure()
        tpm.widget.clear_output(wait=True)
        with tpm.widget:
            display(tpm.fig)
    """

    #with outs:
    tp.promise(run_query, q=f"hello {i}")

In [ ]:
while True:
    sleep(1)
    print(f"done={tp.completed_tasks}, timedout={tp.timedout_tasks}, failed={tp.failed_tasks}, queued={len(tp.futures)}")

In [ ]:
from trulens_eval.appui import AppUI

aui = AppUI(app=ta)
aui.widget